In [1]:
import torch
import numpy as np
from functools import partial
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
import sys
#sys.path.append('./NVIDIA_physicsnemo_examples-weather_corrdiff_DB/datasets')
sys.path.append('/leap/DB_scratch/physicsnemo/research/corrdiff_27_Nov')

In [3]:
from datasets.nyc import NYCDataset

In [4]:
year = 2021

In [5]:
dataset = NYCDataset(
    data_path='/leap/NYC_data_128/4_dec/',
    stats_path='',
    preload=True,
    sel_time=None, #[0, 10],
    file_path='ERA5_hrrr_interp_128_2020_to_2025.nc' ,
    # ERA_only=True,
    sel_year=year)

Using NC files
Preloading data into memory...
Loaded input: (13, 8759, 128, 128), output: (5, 8759, 128, 128)


In [6]:
dataset.time()[0], dataset.time()[-1]

(cftime.DatetimeGregorian(2021, 1, 1, 0, 0, 0, 0, has_year_zero=False),
 cftime.DatetimeGregorian(2021, 12, 31, 23, 0, 0, 0, has_year_zero=False))

In [7]:
dataset.time()[5760], dataset.time()[5860]

(cftime.DatetimeGregorian(2021, 8, 29, 1, 0, 0, 0, has_year_zero=False),
 cftime.DatetimeGregorian(2021, 9, 2, 5, 0, 0, 0, has_year_zero=False))

In [8]:
from physicsnemo.utils.corrdiff import regression_step
from physicsnemo.utils.corrdiff import diffusion_step
from physicsnemo.utils.diffusion import deterministic_sampler, stochastic_sampler
import xarray as xr

def full_generate_func(dataset,  net_reg, net_res, len_to_gen=None, start_gen=0, end_gen=None, num_ensembles=4): 
    # Your generation code here
    if len_to_gen is  None:
        len_to_gen = len(dataset)
    if end_gen is None:
        end_gen = len_to_gen
        
    
    len_to_gen = end_gen - start_gen 
    
    
    num_in_channels = len(dataset.input_channels())
    num_out_channels = len(dataset.output_channels())
    img_shape = dataset.image_shape()

    input_phys_all = np.zeros((len_to_gen, num_in_channels) + img_shape)
    output_phys_all = np.zeros((len_to_gen, num_ensembles, num_out_channels) + img_shape)
    target_phys_all = np.zeros((len_to_gen, num_out_channels) + img_shape)

    for i, idx in enumerate(range(start_gen, end_gen)):
        print('Generating sample {}/{}...'.format(i+1, len_to_gen))
        image_tar, image_lr = dataset[idx]
        image_tar_t = torch.from_numpy(image_tar)
        image_lr_t = torch.from_numpy(image_lr)

        img_lr_batch = image_lr_t.unsqueeze(0).cuda()  # Add batch dim

        num_channels = len(dataset.output_channels())
        seeds = list(range(num_ensembles))

        image_reg = regression_step(
            net=net_reg,
            img_lr=img_lr_batch,
            latents_shape=(1, num_channels, 128, 128),
            lead_time_label=None
        )
        print("Shape of regression image "+str(image_reg.shape))
        
        sampler_fn = partial(stochastic_sampler, patching=None)

        image_res = diffusion_step(
            net=net_res,
            sampler_fn=sampler_fn,
            img_shape=(128, 128),
            img_out_channels=len(dataset.output_channels()),
            rank_batches=[seeds],  # All seeds in one batch
            img_lr=img_lr_batch.expand(num_ensembles, -1, -1, -1),
            rank=0,
            device='cuda',
            mean_hr=image_reg[0:1],  # Use regression as mean
            lead_time_label=None
        )

        output_regression = image_reg
        output_diffusion = image_res
        output_combined = image_reg + image_res


        input_phys_all[i,:,:,:] = dataset.denormalize_input(image_lr_t.cpu().numpy())
        output_phys_all[i,:,:,:,:] = dataset.denormalize_output(output_combined.cpu().numpy())
        target_phys_all[i,:,:,:] = dataset.denormalize_output(image_tar_t.cpu().numpy())



    ds_input_phys = xr.DataArray(input_phys_all, dims=['time', 'variable','y','x'], 
                        coords=dataset.input.isel(time=slice(start_gen, end_gen)).coords).to_dataset(dim='variable')
    ds_output_phys = xr.DataArray(output_phys_all, dims=['time', 'ensemble','variable','y','x'],
                                  coords=dataset.output.isel(time=slice(start_gen, end_gen)).coords).to_dataset(dim='variable')
    ds_target_phys = xr.DataArray(target_phys_all, dims=['time', 'variable','y','x'],
                                 coords=dataset.output.isel(time=slice(start_gen, end_gen)).coords).to_dataset(dim='variable')
    return ds_input_phys, ds_output_phys, ds_target_phys

/usr/local/lib/python3.12/dist-packages/physicsnemo/utils/corrdiff/utils.py:26: ExperimentalFeatureWarning: You are importing from 'physicsnemo.experimental'. The APIs in this namespace are experimental, under active development, and may change without notice. Expect possible back-compatibility breaking changes and only partial test coverage.
  from physicsnemo.experimental import ExperimentalFeatureWarning


In [9]:
from physicsnemo import Module

# Load regression model (deterministic baseline)
net_reg = Module.from_checkpoint("/leap/DB_scratch/physicsnemo/research/corrdiff_27_Nov/results_nyc_3/checkpoints/checkpoints_regression/UNet.0.10000384.mdlus")
net_reg = net_reg.eval().cuda().to(memory_format=torch.channels_last)

# Load diffusion residual model (stochastic refinement)
net_res = Module.from_checkpoint('/leap/DB_scratch/physicsnemo/research/corrdiff_27_Nov/results_nyc_3/checkpoints/checkpoints_diffusion/EDMPrecondSuperResolution.0.10000384.mdlus')
net_res = net_res.eval().cuda().to(memory_format=torch.channels_last)


In [10]:
# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

print(f"Regression model parameters: {count_parameters(net_reg):,}")
print(f"Diffusion model parameters: {count_parameters(net_res):,}")


Regression model parameters: 78,359,045
Diffusion model parameters: 78,364,805


In [11]:
ds_in, ds_out, ds_tar = full_generate_func(dataset, net_reg, net_res, 
                                           start_gen=5760, end_gen=5960, num_ensembles=4)

Generating sample 1/200...


Shape of regression image torch.Size([1, 5, 128, 128])


100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.80s/batch]


Generating sample 2/200...
Shape of regression image torch.Size([1, 5, 128, 128])


100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.10batch/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.18batch/s]


Generating sample 199/200...
Shape of regression image torch.Size([1, 5, 128, 128])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.18batch/s]


Generating sample 200/200...
Shape of regression image torch.Size([1, 5, 128, 128])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.17batch/s]


In [12]:
ds_in['total_precipitation'] = ds_in['total_precipitation']*1000 # convert from m to kg/m^2
# Chaneg the units in attributes as well
ds_in['total_precipitation'].attrs['units'] = 'kg m-2'
common_channels = ['total_column_water_vapour', '2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'total_precipitation']

In [13]:
ds_in.to_netcdf(f'/leap/NYC_data_128/generated_2001/input_{year}_short.nc')
ds_tar.to_netcdf(f'/leap/NYC_data_128/generated_2001/target_{year}_short.nc')
ds_out.to_netcdf(f'/leap/NYC_data_128/generated_2001/output_{year}_short.nc')

In [14]:
ds_in

<xarray.Dataset> Size: 341MB
Dimensions:                    (time: 200, y: 128, x: 128)
Coordinates:
  * time                       (time) datetime64[ns] 2kB 2021-08-29T01:00:00 ...
    atmosphere                 float64 8B 0.0
    gribfile_projection        float64 8B nan
    step                       timedelta64[ns] 8B 00:00:00
    valid_time                 (time) datetime64[ns] 2kB 2021-08-29T01:00:00 ...
    latitude                   (y, x) float64 131kB 39.53 39.53 ... 41.98 41.97
    longitude                  (y, x) float64 131kB 283.6 283.6 ... 289.0 289.1
    surface                    float64 8B 0.0
    valid_time_tp              (time) datetime64[ns] 2kB 2021-08-29T02:00:00 ...
Dimensions without coordinates: y, x
Data variables: (12/13)
    2m_temperature             (time, y, x) float64 26MB 296.7 296.7 ... 293.6
    temperature_500            (time, y, x) float64 26MB 267.0 267.0 ... 263.7
    temperature_850            (time, y, x) float64 26MB 290.0 290.0 ... 286.8
    total_column_water_vapour  (time, y, x) float64 26MB 50.64 50.67 ... 43.67
    total_precipitation        (time, y, x) float64 26MB 0.001287 ... 0.0774
    10m_u_component_of_wind    (time, y, x) float64 26MB -3.062 -3.102 ... 2.462
    ...                         ...
    u_component_of_wind_850    (time, y, x) float64 26MB 0.361 0.2879 ... 10.13
    10m_v_component_of_wind    (time, y, x) float64 26MB -1.296 -1.396 ... 1.51
    v_component_of_wind_500    (time, y, x) float64 26MB -4.367 -4.357 ... 6.631
    v_component_of_wind_850    (time, y, x) float64 26MB -2.886 -2.861 ... 2.607
    geopotential_500           (time, y, x) float64 26MB 5.822e+04 ... 5.636e+04
    geopotential_850           (time, y, x) float64 26MB 1.566e+04 ... 1.428e+04